# Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.utils import shuffle

# Load Data

In [ ]:
data = pd.read_excel('classification-iris.xlsx', header=None).values
# data = pd.read_excel('classification-seeds.xlsx', header=None).values

# Shuffle the data
data = shuffle(data, random_state=42)


num_data = data.shape[0]
num_col = data.shape[1]
num_class = len(np.unique(data[:,num_col-1]))

# Normalize the input data
for ii in range(num_col - 1):
    data[:, ii] = data[:, ii] / np.max(data[:, ii])

# split data
percent_train = 0.7
num_train = round(num_data * percent_train)
num_test = num_data - num_train


# Convert labels to one-hot encoding (necessary for multi-class classification) 
y_one_hot_train = np.zeros((num_train, num_class))
y_one_hot_train[np.arange(num_train), data[:num_train, num_col-1].astype(int)-1] = 1 

y_one_hot_test = np.zeros((num_test, num_class))
y_one_hot_test[np.arange(num_test), data[num_train:, num_col-1].astype(int)-1] = 1

# Set parameters

In [ ]:
# # iris
n0 = data.shape[1]-1
n1 = 10
n2 = 7
n3 = 3
n4 = num_class

# # iris
eta = 0.001
epoch = 500

# # seed
# n0 = data.shape[1]-1
# n1 = 8
# n2 = 15
# n3 = 5
# n4 = num_class

# # seed
# eta = 0.01
# epoch = 60

mse_train = np.zeros((epoch,num_class))
mse_test = np.zeros((epoch,num_class)) 
output_data_train = np.zeros(epoch)
output_data_test = np.zeros(epoch)

a = -1
b = 1

# Initialize weights
np.random.seed(1)
w1 = np.random.uniform(a, b, size=(n1, n0))
w2 = np.random.uniform(a, b, size=(n2, n1))
w3 = np.random.uniform(a, b, size=(n3, n2))
w4 = np.random.uniform(a, b, size=(n4, n3))

# Initialize biases
bias1 = np.random.uniform(a, b, size=(n1, 1))
bias2 = np.random.uniform(a, b, size=(n2, 1))
bias3 = np.random.uniform(a, b, size=(n3, 1))
bias4 = np.random.uniform(a, b, size=(n4, 1))

CorrectiveMatrix = np.ones((num_class,1))

w_par = np.zeros((num_train,n1 * n0 + n1 * 1 + n2 * n1 + n2 * 1 + n3 * n2 + n3 * 1 + n4 * n3 + n4 * 1))
I = np.eye(n1 * n0 + n1 * 1 + n2 * n1 + n2 * 1 + n3 * n2 + n3 * 1 + n4 * n3 + n4 * 1)

# Train and Test

In [ ]:
for t in range(epoch):
    total_error = 0 
    error = np.zeros((num_train, num_class)) 
    
    for i in range(num_train):
        # feed-forward
        input_data = data[i, :num_col-1].reshape(-1,1) 
        net1 = w1 @ input_data + bias1
        o1 = 2/(1 + np.exp(-net1 ** 2))-1 
        net2 = w2 @ o1 + bias2
        o2 = 2 / (1 + np.exp(-net2 ** 2)) -1
        net3 = w3 @ o2 + bias3
        o3 = 2 / (1 + np.exp(-net3 ** 2)) - 1
        net4 = w4 @ o3 + bias4
        o4 =  1 / (1 + np.exp(-net4))  
        
        # Backpropagation for a single sample
        error[i] = (y_one_hot_train[i:i+1]).flatten() - o4.flatten()

        t1 = (1 - o1**2).reshape(-1,1)
        A = np.diag(t1.flatten())

        t2 = (1 - o2**2).reshape(-1,1)
        B = np.diag(t2.flatten())
        
        t3 = (1-o3 ** 2).reshape(-1,1)
        C = np.diag(t3.flatten())
        
        t4 = o4*(1-o4).reshape(-1,1)
        D =np.diag(t4.flatten())
                
        #pw1 = -1  *(o4(1-o4)) * w4*(1-o3**2) * w3 *(1-o2**2) * w2 *(1-o1 **2) * input
        #pw1 = -1 * ((w4.T @ D).T @ C @ w3 @ B @ w2 @ A).T @ input_data
        #CorrectiveMatrix = np.array([[1], [1], [1]])
        pw1 = -1 * (((w4.T @ D).T @ C @ w3 @ B @ w2 @ A).T @ CorrectiveMatrix) @ input_data.T  
        pb1 = -1 * (((w4.T @ D).T @ C @ w3 @ B @ w2 @ A).T @ CorrectiveMatrix)
        
        # -1 * F’(net4) * w4 * F’(net3)* w3 * F’(net2) * o1 =-1 * *(o4(1-o4)) * w4*(1-o3**2) * w3 *(1-o2**2) * o1
        pw2 = -1 * (((w4.T @ D).T  @ C @ w3 @ B).T @ CorrectiveMatrix  @ o1.T)
        pb2 = -1 * (((w4.T @ D).T  @ C @ w3 @ B).T @ CorrectiveMatrix)
    
        # -1*(o4(1-o4)) * w4*(1-o3**2) *o2
        pw3 = -1 * ((w4.T @ D).T @  C).T  @ CorrectiveMatrix @ o2.T
        pb3 = -1 * ((w4.T @ D).T @  C).T  @ CorrectiveMatrix 
        
        #-1 * F’(net4) * o3 =  -1* o4(1-o4) * o3 
        pw4 = -1 *  (D @ CorrectiveMatrix @ o3.T)
        pb4 = -1 *  (D @ CorrectiveMatrix)
                
        a = pw1.flatten()
        b = pb1.flatten()
        c = pw2.flatten()
        d = pb2.flatten()
        e = pw3.flatten()
        f = pb3.flatten()
        g = pw4.flatten()
        h = pb4.flatten()
        
        w_par[i, :] = np.concatenate((a, b, c, d, e, f, g, h))
                
        total_error += np.abs(error[i]) 

    a1 = w1.reshape(-1,1)
    b1 = bias1.reshape(-1,1)
    c1 = w2.reshape(-1,1)
    d1 = bias2.reshape(-1,1)
    e1 = w3.reshape(-1,1)
    f1 = bias3.reshape(-1,1)
    g1 = w4.reshape(-1,1)
    h1 = bias4.reshape(-1,1)

    w_par1 = np.concatenate((a1, b1, c1, d1, e1, f1, g1, h1)).reshape(-1,1)

    miu = (error.T @ error).flatten()[0]
 
    w_par1 = w_par1 - np.linalg.inv((w_par.T @ w_par) + miu * I) @ w_par.T @ (error @ CorrectiveMatrix)
    
    a2 = w_par1[:n1 * n0].reshape((n1, n0))
    b2 = w_par1[n1 * n0:n1 * n0 + n1 * 1].reshape((n1,1))
    c2 = w_par1[n1 * n0 + n1 * 1: n1 * n0 + n1 * 1 + n2 * n1].reshape((n2, n1))
    d2 =  w_par1[ n1 * n0 + n1 * 1 + n2 * n1: n1 * n0 + n1 * 1 + n2 * n1 + n2 * 1].reshape((n2, 1))
    e2 =  w_par1[ n1 * n0 + n1 * 1 + n2 * n1 + n2 * 1: n1 * n0 + n1 * 1 + n2 * n1 + n2 * 1 + n3 * n2].reshape((n3, n2))
    f2 =  w_par1[ n1 * n0 + n1 * 1 + n2 * n1 + n2 * 1 + n3 * n2: n1 * n0 + n1 * 1 + n2 * n1 + n2 * 1 + n3 * n2 + n3 * 1].reshape((n3, 1))
    g2 =  w_par1[ n1 * n0 + n1 * 1 + n2 * n1 + n2 * 1 + n3 * n2 + n3 * 1: n1 * n0 + n1 * 1 + n2 * n1 + n2 * 1 + n3 * n2 + n3 * 1 + n4 * n3].reshape((n4, n3))
    h2 =  w_par1[ n1 * n0 + n1 * 1 + n2 * n1 + n2 * 1 + n3 * n2 + n3 * 1 + n4 * n3:].reshape((n4, 1))

    w1 = a2
    bias1 = b2
    w2 = c2
    bias2 = d2
    w3 = e2
    bias3 = f2
    w4 = g2
    bias4 = h2

    error_data_train = np.zeros((num_train, num_class))
    output_data_train = np.zeros(num_train)
  
    for i in range(num_train):
        input_data = data[i, :num_col-1]
        input_data = input_data.reshape(-1,1)
        net1 = w1 @ input_data #(30, 4)x(4,1) = 30 x 1
        
        o1 = 2/(1 + np.exp(-net1 ** 2))-1 
        net2 = w2 @ o1
        o2 = 2 / (1 + np.exp(-net2 ** 2)) -1
        net3 = w3 @ o2
        o3 = 2 / (1 + np.exp(-net3 ** 2)) - 1
        net4 = w4 @ o3
        o4 =  1 / (1 + np.exp(-net4))
#         o4 = net4

        output_data_train[i] = np.argmax(o4)+1 
        error_data_train[i] = (y_one_hot_train[i]).flatten() - o4.flatten()

    mse_train[t] = np.mean(error_data_train ** 2,axis=0)

    error_data_test = np.zeros((num_test, num_class))
    output_data_test = np.zeros(num_test)
    
    
    for i in range(num_test):
        input_data = (data[num_train + i, :num_col-1]).reshape(-1,1) 
        net1 = w1 @ input_data  
        o1 = 2/(1 + np.exp(-net1 ** 2))-1 
        net2 = w2 @ o1
        o2 = 2 / (1 + np.exp(-net2 ** 2)) -1
        net3 = w3 @ o2
        o3 = 2 / (1 + np.exp(-net3 ** 2)) - 1
        net4 = w4 @ o3
        o4 =  1 / (1 + np.exp(-net4))
#         o4 = net4

        output_data_test[i] = np.argmax(o4)+1
    
        error_data_test[i] = (y_one_hot_test[i:i+1]).flatten() - o4.flatten()

    mse_test[t] = np.mean(error_data_test ** 2,axis=0)

    plt.figure(1)
    plt.subplot(2, 2, 1)
    plt.plot(data[:num_train, num_col-1], '-sr')
    plt.plot(output_data_train, '-*b')
    plt.xlabel('Train Data')
    plt.ylabel('Output')   
    
#         Plotting the training MSE
    plt.subplot(2, 2, 2)
    plt.semilogy(np.arange(1, t + 1), mse_train[:t])
    plt.xlabel('Epoch')
    plt.ylabel('MSE Train')
    

    plt.subplot(2, 2, 3)    
    plt.plot(data[num_train:, num_col-1], '-sr')
    plt.plot(output_data_test, '-*b')
    plt.xlabel('Test Data')
    plt.ylabel('Output')
    
        # Plotting the test MSE
    plt.subplot(2, 2, 4)
    plt.semilogy(np.arange(1, t + 1), mse_test[:t])
    plt.xlabel('Epoch')
    plt.ylabel('MSE Test')

    
    print('Epoch: {} \t'.format(t+1))
    print('MSE_train: ',mse_train[t],' MSE_Test: ' ,mse_test[t])
    
    plt.tight_layout()
    plt.show()
    
    print("\n\033[1;m" + "*" * 125)
    
    
plt.figure(2)
m_train , b_train = np.polyfit(data[:num_train, num_col - 1], output_data_train, 1)    
plt.scatter(data[:num_train, num_col - 1], output_data_train,facecolors='none',edgecolors='#104E8B')
plt.plot(data[:num_train, num_col - 1], m_train*data[:num_train, num_col - 1]+b_train,'r')
# plt.plot(data[:num_train, 3], output_data_train, label='Regression Line', color='red')
plt.title('Regression Train') 

plt.figure(3)
m_test , b_test = np.polyfit(data[num_train:, num_col - 1], output_data_test, 1)  
plt.scatter(data[num_train:, num_col - 1], output_data_test,facecolors='none',edgecolors='#104E8B')
plt.plot(data[num_train:, num_col - 1], m_test*data[num_train:,num_col - 1]+b_test,'r')
plt.title('Regression Test')
 
mse_train_result = mse_train[-1]
mse_test_result = mse_test[-1]

print("Final MSE on Train Data:", mse_train_result)
print("Final MSE on Test Data:", mse_test_result)


In [ ]:

# Assuming you have converted regression predictions to classes
predicted_classes = output_data_train 
actual_classes = data[:num_train,num_col-1] 

# Define the number of classes

# Create a confusion matrix-like matrix
confusion_matrix = np.zeros((num_class, num_class))

# Fill the confusion matrix
for actual, predicted in zip(actual_classes, predicted_classes):
    confusion_matrix[actual.astype(int)-1][predicted.astype(int)-1] += 1

# Plot the confusion matrix
plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Train Confusion Matrix ')
plt.colorbar()

# Annotate the plot with numbers
for i in range(num_class):
    for j in range(num_class):
        plt.text(j, i, str(int(confusion_matrix[i, j])), fontsize=12, ha='center', va='center')  # Corrected

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(np.arange(num_class), np.arange(1, num_class + 1))
plt.yticks(np.arange(num_class), np.arange(1, num_class + 1))

plt.show()


In [ ]:
# Assuming you have converted regression predictions to classes
predicted_classes = output_data_test 
actual_classes = data[num_train:,num_col-1] 

# Define the number of classes

# Create a confusion matrix-like matrix
confusion_matrix = np.zeros((num_class, num_class))

# Fill the confusion matrix
for actual, predicted in zip(actual_classes, predicted_classes):
    confusion_matrix[actual.astype(int)-1][predicted.astype(int)-1] += 1

# Plot the confusion matrix
plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Test Confusion Matrix ')
plt.colorbar()

# Annotate the plot with numbers
for i in range(num_class):
    for j in range(num_class):
        plt.text(j, i, str(int(confusion_matrix[i, j])), fontsize=12, ha='center', va='center')  # Corrected
# Adjust ticks and labels to start from 1
        
plt.xlabel('Predicted')
plt.ylabel('Actual')
# plt.xticks(np.arange(num_class))
# plt.yticks(np.arange(num_class))
plt.xticks(np.arange(num_class), np.arange(1, num_class + 1))
plt.yticks(np.arange(num_class), np.arange(1, num_class + 1))

plt.show()



In [ ]:
train_accuracy = np.mean(output_data_train == data[:num_train,num_col-1] ) 
print(f"Accuracy on the train set: {train_accuracy * 100}%")

test_accuracy = np.mean(output_data_test == data[num_train:,num_col-1]) 
print(f"Accuracy on the test set: {test_accuracy * 100}%")